In [43]:
from typing import List

from langchain.document_loaders.unstructured import UnstructuredFileLoader
from paddleocr import PaddleOCR
import os
import fitz
import nltk
# from configs.model_config import NLTK_DATA_PATH

NLTK_DATA_PATH = './nltk_data'

nltk.data.path = [NLTK_DATA_PATH] + nltk.data.path


In [44]:
class UnstructuredPaddlePDFLoader(UnstructuredFileLoader):
    def __get_elements(self)->List:
        def pdf_or_txt(filepath,dir_path = 'tmp_files'):
            full_dir_path = os.path.join(os.path.dirname(filepath), dir_path)
            if not os.path.exists(full_dir_path):
                os.makedirs(full_dir_path)
            ocr = PaddleOCR(use_angle_cls=True, lang="ch", use_gpu=False, show_log=False)
            doc = fitz.open(filepath)
            txt_file_path = os.path.join(full_dir_path, f"{os.path.split(filepath)[-1]}.txt")
            img_name = os.path.join(full_dir_path, 'tmp.png')
            with open(txt_file_path,'w',encoding='utf-8') as fout:
                for i in range(doc.page_count):
                    page = doc[i]
                    text = page.get_text("")
                    fout.write(text)
                    fout.write('\n')

                    img_list = page.get_image()
                    for img in img_list:
                        pix = fitz.Pixmap(doc, img[0])
                        if pix.n - pix.alpha >=4:
                            pix = fitz.Pixmap(fitz.csRGB, pix)
                        pix.save(img_name)
                        
                        result = ocr.ocr(img_name)
                        ocr_result = [i[1][0] for line in result for i in line]
                        fout.write('\n'.join(ocr_result))
            if os.path.exists(img_name):
                os.remove(img_name)
            return txt_file_path

        txt_file_path = pdf_or_txt(self.file_path)
        from unstructured.partition.text import partition_text
        return partition_text(file_name = txt_file_path,**self.unstructured_kwargs)

In [53]:
filepath =  "./database/VisualGLM微调.pdf"
loader = UnstructuredPaddlePDFLoader(filepath)
docs = loader.load()
for doc in docs:
    print(doc)

[2023-07-03 15:44:26,737] [ WARNING] pdf.py:149 - detectron2 is not installed. Cannot use the hi_res partitioning strategy. Falling back to partitioning with the fast strategy.


page_content='VisualGLM\n\n\n\n6B的原理与微调\n\n报告⼈：丁    铭\n\n清华⼤学计算机系\n\n智谱AI\n\nhttps://github.com/THUDM/\n\nVisualGLM\n\n\n\n6B\n\n大纲\n\n多模态预训练背景\n\n– 多模态预训练思路一（CogView思路）\n\n图像作为一种特殊的语言\n\n– 多模态预训练思路二（VisualGLM思路）\n\n图像对齐到预训练语言模型\n\n– ChatGLM\n\n\n\n6B的简单介绍\n\nVisualGLM\n\n– 训练过程\n\n– 安装与使用\n\n– 使用sat库进行微调\n\n未来展望\n\n2\n\n背景：多模态预训练\n\n以BERT/GPT为代表的自回归模型展现出强大的理解生成能力\n\n如何将这种成功“复制”到视觉/跨模态领域？\n\n以DALL-E、CogView、BEiT等为代表的模型的思路为：\n\n– 将图像当成一种语言进行预训练\n\n– 离散化\x00token\n\n优势：\n\n– 图像为一等公民，同时进行图像生成和理解\n\n劣势：\n\ncogview\n\n– 离散化有损，降低token利用率，需要\x00>1000tokens\x00来描述一张256分辨率的图\n\n3\n\n背景：多模态预训练\n\n实际上，人类只对少量的视觉语义信息感兴趣\n\n如何提升效率，并充分利用语言模型？\n\nBLIP2提供了一种思路：\n\n– 将图像特征对齐到预训练语言模型\n\n优势：\n\n– 充分利用语言模型\n\n– 无缝接合原有的多轮对话能力\n\n劣势：\n\n– 提取图像语义特征损失底层信息\n\n4\n\n背景：ChatGLM\n\n\n\n6B\n\nChatGLM-6B 是一个开源的、具有62亿参数的中英双语语言模型。\n\n自3月14日发布之后一个月内，在 Github\x00上获得了\x002.1\x00万\x00star，全球\n\n下载量超过\x00100\x00万，登上Github Trending\x00榜第一、Hugging Face\n\nTrending\x00榜第一。\n\n海量中英文本\n\n文本预训练\n\nGLM\n\n\n\n130B\n\n指示\n\n\n\n回复配

In [54]:
import re
docs[0].page_content =re.sub(r"\n{1,}", r"", docs[0].page_content)
docs[0].page_content =re.sub(r"\s", r"", docs[0].page_content)

In [55]:
docs[0].page_content

'VisualGLM6B的原理与微调报告⼈：丁铭清华⼤学计算机系智谱AIhttps://github.com/THUDM/VisualGLM6B大纲多模态预训练背景–多模态预训练思路一（CogView思路）图像作为一种特殊的语言–多模态预训练思路二（VisualGLM思路）图像对齐到预训练语言模型–ChatGLM6B的简单介绍VisualGLM–训练过程–安装与使用–使用sat库进行微调未来展望2背景：多模态预训练以BERT/GPT为代表的自回归模型展现出强大的理解生成能力如何将这种成功“复制”到视觉/跨模态领域？以DALL-E、CogView、BEiT等为代表的模型的思路为：–将图像当成一种语言进行预训练–离散化\x00token优势：–图像为一等公民，同时进行图像生成和理解劣势：cogview–离散化有损，降低token利用率，需要\x00>1000tokens\x00来描述一张256分辨率的图3背景：多模态预训练实际上，人类只对少量的视觉语义信息感兴趣如何提升效率，并充分利用语言模型？BLIP2提供了一种思路：–将图像特征对齐到预训练语言模型优势：–充分利用语言模型–无缝接合原有的多轮对话能力劣势：–提取图像语义特征损失底层信息4背景：ChatGLM6BChatGLM-6B是一个开源的、具有62亿参数的中英双语语言模型。自3月14日发布之后一个月内，在Github\x00上获得了\x002.1\x00万\x00star，全球下载量超过\x00100\x00万，登上GithubTrending\x00榜第一、HuggingFaceTrending\x00榜第一。海量中英文本文本预训练GLM130B指示回复配对数据指令微调反馈自助人类偏好数据人类反馈强化学习ChatGLM5VisualGLM6B如何让ChatGLM拥有图像识别的能力？通用领域中英双语与正常语言问答顺畅融合基本思路：通过中间模块构建起预训练视觉和语言模型的桥梁中英双语图文数据大规模预训练高质量指令数据微调与纯语言类似的技术方案BLIP2、MiniGPT4等工作类似思路6VisualGLM6B\x00训练模型架构：•ViT+QFormer+ChatGLM-6B•几乎冻结ViT\x00和\x00ChatGLM\x00参数防止灾难性遗忘预训练：QFormer\x00和\x00VitLora进行学习微调：Q

In [56]:
docs

[Document(page_content='VisualGLM6B的原理与微调报告⼈：丁铭清华⼤学计算机系智谱AIhttps://github.com/THUDM/VisualGLM6B大纲多模态预训练背景–多模态预训练思路一（CogView思路）图像作为一种特殊的语言–多模态预训练思路二（VisualGLM思路）图像对齐到预训练语言模型–ChatGLM6B的简单介绍VisualGLM–训练过程–安装与使用–使用sat库进行微调未来展望2背景：多模态预训练以BERT/GPT为代表的自回归模型展现出强大的理解生成能力如何将这种成功“复制”到视觉/跨模态领域？以DALL-E、CogView、BEiT等为代表的模型的思路为：–将图像当成一种语言进行预训练–离散化\x00token优势：–图像为一等公民，同时进行图像生成和理解劣势：cogview–离散化有损，降低token利用率，需要\x00>1000tokens\x00来描述一张256分辨率的图3背景：多模态预训练实际上，人类只对少量的视觉语义信息感兴趣如何提升效率，并充分利用语言模型？BLIP2提供了一种思路：–将图像特征对齐到预训练语言模型优势：–充分利用语言模型–无缝接合原有的多轮对话能力劣势：–提取图像语义特征损失底层信息4背景：ChatGLM6BChatGLM-6B是一个开源的、具有62亿参数的中英双语语言模型。自3月14日发布之后一个月内，在Github\x00上获得了\x002.1\x00万\x00star，全球下载量超过\x00100\x00万，登上GithubTrending\x00榜第一、HuggingFaceTrending\x00榜第一。海量中英文本文本预训练GLM130B指示回复配对数据指令微调反馈自助人类偏好数据人类反馈强化学习ChatGLM5VisualGLM6B如何让ChatGLM拥有图像识别的能力？通用领域中英双语与正常语言问答顺畅融合基本思路：通过中间模块构建起预训练视觉和语言模型的桥梁中英双语图文数据大规模预训练高质量指令数据微调与纯语言类似的技术方案BLIP2、MiniGPT4等工作类似思路6VisualGLM6B\x00训练模型架构：•ViT+QFormer+ChatGLM-6B•几乎冻结ViT\x00和\x00ChatGLM\x00参数防止灾难性遗忘预训练：QFormer\

In [57]:
from textsplitter import ChineseTextSplitter

In [58]:
sentence_size = 500

In [59]:
textsplitter = ChineseTextSplitter(pdf=True, sentence_size=sentence_size)
docs = loader.load_and_split(textsplitter)

[2023-07-03 15:44:38,459] [ WARNING] pdf.py:149 - detectron2 is not installed. Cannot use the hi_res partitioning strategy. Falling back to partitioning with the fast strategy.


In [52]:
docs

[Document(page_content='VisualGLM 6B的原理与微调  报告⼈：丁    铭  清华⼤学计算机系  智谱AI  https://github.', metadata={'source': './database/VisualGLM微调.pdf'}),
 Document(page_content='com/THUDM/  VisualGLM 6B  大纲  多模态预训练背景  – 多模态预训练思路一（CogView思路）  图像作为一种特殊的语言  – 多模态预训练思路二（VisualGLM思路）  图像对齐到预训练语言模型  – ChatGLM 6B的简单介绍  VisualGLM  – 训练过程  – 安装与使用  – 使用sat库进行微调  未来展望  2  背景：多模态预训练  以BERT/GPT为代表的自回归模型展现出强大的理解生成能力  如何将这种成功“复制”到视觉/跨模态领域？', metadata={'source': './database/VisualGLM微调.pdf'}),
 Document(page_content='  以DALL-E、CogView、BEiT等为代表的模型的思路为：  – 将图像当成一种语言进行预训练  – 离散化\x00token  优势：  – 图像为一等公民，同时进行图像生成和理解  劣势：  cogview  – 离散化有损，降低token利用率，需要\x00>1000tokens\x00来描述一张256分辨率的图  3  背景：多模态预训练  实际上，人类只对少量的视觉语义信息感兴趣  如何提升效率，并充分利用语言模型？', metadata={'source': './database/VisualGLM微调.pdf'}),
 Document(page_content='  BLIP2提供了一种思路：  – 将图像特征对齐到预训练语言模型  优势：  – 充分利用语言模型  – 无缝接合原有的多轮对话能力  劣势：  – 提取图像语义特征损失底层信息  4  背景：ChatGLM 6B  ChatGLM-6B 是一个开源的、具有62亿参数的中英双语语言模型。', metadata={'source': './database/VisualGLM微调.pdf'}),
 Doc

In [5]:
filepath = input("Input your local knowledge folder path 请输入本地知识文件夹路径：\n")

In [18]:
filepath = './database/'

In [46]:
import glob
filepath = './database/'
file_list = [i for j in ['txt','pdf','md','csv'] for i in glob.glob(filepath +"*."+j) ]

In [47]:
file_list

['./database/test.txt', './database/VisualGLM微调.pdf', './database/test.pdf']